# Reddit Rule Violation - XGBoost 성능 최적화 모델

이 노트북은 EDA에서 얻은 인사이트를 바탕으로, 성능을 최대한 끌어올리기 위한 XGBoost 모델을 구현합니다.

**주요 전략:**
1. **피처 엔지니어링**: 기존 텍스트 스타일 피처 + `subreddit` 타겟 인코딩
2. **텍스트 벡터화**: TF-IDF (Word + Character n-grams)
3. **모델**: XGBoost Classifier
4. **검증**: Stratified 5-Fold Cross-Validation

## 1. 라이브러리 임포트 및 데이터 로드

In [13]:
import pandas as pd
import numpy as np
import re
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from scipy.sparse import hstack

warnings.filterwarnings('ignore')

# 데이터 로드
train_df = pd.read_csv('train.csv')

## 2. 피처 엔지니어링 함수 정의

EDA에서 사용했던 텍스트 스타일 피처 생성 함수들을 정의합니다.

In [14]:
def add_text_features(df):
    df['body_len'] = df['body'].apply(len)
    df['url_cnt'] = df['body'].apply(lambda x: len(re.findall(r'http\S+', x)))
    df['exc_cnt'] = df['body'].apply(lambda x: x.count('!'))
    df['q_cnt'] = df['body'].apply(lambda x: x.count('?'))
    df['upper_rt'] = df['body'].apply(lambda x: len([c for c in x if c.isupper()]) / (len(x) + 1e-6))
    return df

train_df = add_text_features(train_df)

## 3. 모델 학습 및 교차 검증

Stratified K-Fold를 사용하여 교차 검증을 수행합니다. 각 Fold 내부에서 **타겟 인코딩**을 수행하여 데이터 누설(Leakage)을 방지합니다.

In [15]:
target = 'rule_violation'
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

oof_preds = np.zeros(len(train_df))
auc_scores = []

# 텍스트 입력 생성
train_df['text_input'] = train_df['rule'] + " [SEP] " + train_df['body']

for fold, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df[target])):
    print(f"--- Fold {fold+1} ---")
    
    # 데이터 분리
    X_train, X_val = train_df.iloc[train_idx], train_df.iloc[val_idx]
    y_train, y_val = X_train[target], X_val[target]
    
    # 1. 타겟 인코딩 (폴드 내에서 수행)
    subreddit_map = y_train.groupby(X_train['subreddit']).mean()
    X_train['subreddit_encoded'] = X_train['subreddit'].map(subreddit_map)
    X_val['subreddit_encoded'] = X_val['subreddit'].map(subreddit_map)
    X_train['subreddit_encoded'].fillna(y_train.mean(), inplace=True)
    X_val['subreddit_encoded'].fillna(y_train.mean(), inplace=True)
    
    # 2. TF-IDF 벡터화
    word_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_features=10000)
    char_vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4), min_df=3, max_features=10000)
    
    X_train_text_word = word_vectorizer.fit_transform(X_train['text_input'])
    X_val_text_word = word_vectorizer.transform(X_val['text_input'])
    
    X_train_text_char = char_vectorizer.fit_transform(X_train['text_input'])
    X_val_text_char = char_vectorizer.transform(X_val['text_input'])
    
    # 3. 피처 결합
    numeric_features = ['body_len', 'url_cnt', 'exc_cnt', 'q_cnt', 'upper_rt', 'subreddit_encoded']
    X_train_numeric = X_train[numeric_features].values
    X_val_numeric = X_val[numeric_features].values
    
    X_train_combined = hstack([X_train_text_word, X_train_text_char, X_train_numeric])
    X_val_combined = hstack([X_val_text_word, X_val_text_char, X_val_numeric])
    
    # 4. XGBoost 모델 학습
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        n_estimators=1000,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.7,
        colsample_bytree=0.7,
        use_label_encoder=False,
        random_state=42,
        early_stopping_rounds=50, #<- 위치 이동
        # GPU 사용 시 주석 해제
        tree_method='gpu_hist' 
    )
    
    model.fit(X_train_combined, y_train,
              eval_set=[(X_val_combined, y_val)],
              verbose=100)
    
    val_preds = model.predict_proba(X_val_combined)[:, 1]
    oof_preds[val_idx] = val_preds
    auc = roc_auc_score(y_val, val_preds)
    auc_scores.append(auc)
    print(f"[Fold {fold+1}] AUC = {auc:.4f}")

print("\n--- 최종 결과 ---")
print(f"CV AUC: {np.mean(auc_scores):.4f} +/- {np.std(auc_scores):.4f}")

--- Fold 1 ---
[0]	validation_0-auc:0.72676
[100]	validation_0-auc:0.83174
[166]	validation_0-auc:0.83450
[Fold 1] AUC = 0.8353
--- Fold 2 ---
[0]	validation_0-auc:0.61144
[100]	validation_0-auc:0.77483
[200]	validation_0-auc:0.78701
[300]	validation_0-auc:0.79005
[308]	validation_0-auc:0.79012
[Fold 2] AUC = 0.7913
--- Fold 3 ---
[0]	validation_0-auc:0.68424
[100]	validation_0-auc:0.82222
[200]	validation_0-auc:0.83171
[300]	validation_0-auc:0.83824
[358]	validation_0-auc:0.83732
[Fold 3] AUC = 0.8390
--- Fold 4 ---
[0]	validation_0-auc:0.64249
[100]	validation_0-auc:0.78982
[200]	validation_0-auc:0.80116
[300]	validation_0-auc:0.80150
[318]	validation_0-auc:0.80179
[Fold 4] AUC = 0.8057
--- Fold 5 ---
[0]	validation_0-auc:0.68920
[100]	validation_0-auc:0.81500
[200]	validation_0-auc:0.82302
[300]	validation_0-auc:0.82618
[327]	validation_0-auc:0.82635
[Fold 5] AUC = 0.8287

--- 최종 결과 ---
CV AUC: 0.8200 +/- 0.0184


## 4. (참고) 테스트 데이터 예측 및 제출 파일 생성

실제 대회 제출을 위해서는 전체 학습 데이터로 모델을 재학습하고 테스트 데이터에 대해 예측해야 합니다. 아래는 그 과정을 담은 예시 코드입니다.

In [ ]:
def make_submission(train_df, test_df):
    print("전체 데이터로 재학습 및 예측을 시작합니다...")
    
    # 피처 엔지니어링
    train_df = add_text_features(train_df)
    test_df = add_text_features(test_df)
    
    # 텍스트 입력 생성
    train_df['text_input'] = train_df['rule'] + " [SEP] " + train_df['body']
    test_df['text_input'] = test_df['rule'] + " [SEP] " + test_df['body']
    
    # 타겟 인코딩 (전체 학습 데이터 기준)
    subreddit_map = train_df.groupby('subreddit')[target].mean()
    train_df['subreddit_encoded'] = train_df['subreddit'].map(subreddit_map)
    test_df['subreddit_encoded'] = test_df['subreddit'].map(subreddit_map)
    train_df['subreddit_encoded'].fillna(train_df[target].mean(), inplace=True)
    test_df['subreddit_encoded'].fillna(train_df[target].mean(), inplace=True)

    # TF-IDF 벡터화
    word_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3, max_features=10000)
    char_vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4), min_df=3, max_features=10000)

    X_train_text_word = word_vectorizer.fit_transform(train_df['text_input'])
    X_test_text_word = word_vectorizer.transform(test_df['text_input'])
    
    X_train_text_char = char_vectorizer.fit_transform(train_df['text_input'])
    X_test_text_char = char_vectorizer.transform(test_df['text_input'])
    
    # 피처 결합
    numeric_features = ['body_len', 'url_cnt', 'exc_cnt', 'q_cnt', 'upper_rt', 'subreddit_encoded']
    X_train_numeric = train_df[numeric_features].values
    X_test_numeric = test_df[numeric_features].values

    X_train_combined = hstack([X_train_text_word, X_train_text_char, X_train_numeric])
    X_test_combined = hstack([X_test_text_word, X_test_text_char, X_test_numeric])
    
    # XGBoost 모델 학습
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        n_estimators=1200, # early stopping 최적값 근사치로 설정
        learning_rate=0.05,
        max_depth=6,
        subsample=0.7,
        colsample_bytree=0.7,
        use_label_encoder=False,
        random_state=42
    )
    
    model.fit(X_train_combined, train_df[target])
    
    # 예측 및 제출 파일 생성
    predictions = model.predict_proba(X_test_combined)[:, 1]
    submission_df = pd.DataFrame({'row_id': test_df['row_id'], 'rule_violation': predictions})
    submission_df.to_csv('submission.csv', index=False)
    print("\n제출 파일 'submission.csv'가 생성되었습니다.")
    return submission_df




In [17]:
# test.csv 파일이 있다면 아래 코드의 주석을 해제하여 실행하세요.
test_df = pd.read_csv('test.csv')
submission = make_submission(train_df.copy(), test_df.copy()) 
print(submission.head())

전체 데이터로 재학습 및 예측을 시작합니다...


ValueError: Must have at least 1 validation dataset for early stopping.